In [ ]:
!pip install langgraph==0.2.23 langgraph-checkpoint==1.0.10 langchain_core==0.3.0 httpx==0.27.2

In [ ]:
from typing import Any, Optional
from pydantic import BaseModel

# グラフの状態を定義
class State(BaseModel):
    count: Optional[int]
    message: str

# 簡単なノード関数を定義
def increment_count(state: State) -> dict[str, Any]:
    if state.count is None:
        state.count = 0
    return {"count": state.count + 1}

def update_message(state: State) -> dict[str, Any]:
    return {"message": f"カウントは {state.count}"}

In [ ]:
import json
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)
    checkpoint = checkpoint_tuple.checkpoint
    metadata = checkpoint_tuple.metadata

    print("チェックポイントデータ:")
    print(json.dumps(checkpoint, indent=2, ensure_ascii=False))
    print("\nメタデータ:")
    print(json.dumps(metadata, indent=2, ensure_ascii=False))

In [ ]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

# グラフを設定
graph = StateGraph(State)
graph.add_node("increment", increment_count)
graph.add_node("update", update_message)

graph.set_entry_point("increment")
graph.add_edge("increment", "update")
graph.add_edge("update", "increment")

# チェックポインターを設定
checkpointer = MemorySaver()

# グラフをコンパイル
compiled_graph = graph.compile(checkpointer=checkpointer, interrupt_after=["update"])

In [ ]:
# グラフを実行
config = {"configurable": {"thread_id": "example-thread"}}
initial_state = State(count=0, message="initial")
first_state = compiled_graph.invoke(initial_state, config)
first_state

In [ ]:
for checkpoint_tuple in checkpointer.list({"configurable": {"thread_id": "example-thread"}}):
    print(checkpoint_tuple.checkpoint["channel_values"])

In [ ]:
# チェックポイントデータを取得
print_checkpoint_dump(checkpointer, config)

In [ ]:
second_state = compiled_graph.invoke(None, config)
second_state

In [ ]:
for checkpoint_tuple in checkpointer.list({"configurable": {"thread_id": "example-thread"}}):
    print(checkpoint_tuple.checkpoint['channel_values'])

In [ ]:
# チェックポイントデータを取得
print_checkpoint_dump(checkpointer, config)